In [9]:
import pandas as pd
import numpy as np
from matplotlib import pyplot

In [10]:
board_game_set = pd.read_csv('data/secondBoardGameDataSet.csv')
board_game_set.head(5)

,@objectid,name,yearpublished,boardgamecategory,minplayers,maxplayers,age,minplaytime,maxplaytime,description,image
0,1,Die Macher,1986.0,"['Economic', 'Negotiation', 'Political']",3.0,5.0,14.0,240.0,240.0,Die Macher is a game about seven sequential po...,https://cf.geekdo-images.com/rpwCZAjYLD940NWwP...
1,2,Dragonmaster,1981.0,"['Card Game', 'Fantasy']",3.0,4.0,12.0,30.0,30.0,Dragonmaster is a trick-taking card game based...,https://cf.geekdo-images.com/oQYhaJx5Lg3KcGis2...
2,3,Samouraï,1998.0,"['Abstract Strategy', 'Medieval']",2.0,4.0,10.0,30.0,60.0,Samurai is set in medieval Japan. Players comp...,https://cf.geekdo-images.com/o9-sNXmFS_TLAb7Zl...
3,4,Tal der Könige,1992.0,['Ancient'],2.0,4.0,12.0,60.0,60.0,When you see the triangular box and the luxuri...,https://cf.geekdo-images.com/nYiYhUlatT2DpyXaJ...
4,5,Acquire,1963.0,"['Economic', 'Territory Building']",2.0,6.0,12.0,90.0,90.0,"In Acquire, each player strategically invests ...",https://cf.geekdo-images.com/FfguJeknahk88vKT7...


In [11]:
all_categories = board_game_set['boardgamecategory']
all_categories_list = [categories.strip("['']").split(', ') if isinstance(categories, str) else [] for categories in all_categories]
all_categories_list[0]

["Economic'", "'Negotiation'", "'Political"]

In [12]:
singular_game_categories = []

for index, categories in enumerate(all_categories_list):
    clean_categories = []
    for category in categories:
        category = category.strip('"')
        category = category.strip("'")
        category = category.replace("'", '')
        category = category.replace('"', '')
        clean_categories.append(category)
        if category not in singular_game_categories:
            singular_game_categories.append(category)
    all_categories_list[index] = clean_categories

singular_game_categories

['Economic',
 'Negotiation',
 'Political',
 'Card Game',
 'Fantasy',
 'Abstract Strategy',
 'Medieval',
 'Ancient',
 'Territory Building',
 'Civilization',
 'Nautical',
 'Childrens Game',
 'City Building',
 'Exploration',
 'Travel',
 'Farming',
 'Mythology',
 'Bluffing',
 'Science Fiction',
 'Collectible Components',
 'Dice',
 'Fighting',
 'Print & Play',
 'Maze',
 'Miniatures',
 'Racing',
 'American West',
 'Wargame',
 'Adventure',
 'Space Exploration',
 'Renaissance',
 'Modern Warfare',
 'Humor',
 'Electronic',
 'Horror',
 'Novel-based',
 'Deduction',
 'Word Game',
 'Aviation / Flight',
 'Movies / TV / Radio theme',
 'Party Game',
 'Memory',
 'Math',
 'Puzzle',
 'Real-time',
 'Trivia',
 'Industry / Manufacturing',
 'World War II',
 'Expansion for Base-game',
 'American Civil War',
 'Age of Reason',
 'American Indian Wars',
 'World War I',
 'Trains',
 'Animals',
 'Pirates',
 'Murder/Mystery',
 'Transportation',
 'Prehistoric',
 'Sports',
 'Action / Dexterity',
 'Game System',
 'Number

In [13]:
board_game_set['boardgamecategory'] = all_categories_list

In [14]:
board_game_set['boardgamecategory']

0         [Economic, Negotiation, Political]
1                       [Card Game, Fantasy]
2              [Abstract Strategy, Medieval]
3                                  [Ancient]
4             [Economic, Territory Building]
                         ...                
186108                                    []
186109                                    []
186110                                    []
186111                                    []
186112                                    []
Name: boardgamecategory, Length: 186113, dtype: object

In [15]:
for singular_game_category in singular_game_categories:
    board_game_set[singular_game_category] = 0

board_game_set

,@objectid,name,yearpublished,boardgamecategory,minplayers,maxplayers,age,minplaytime,maxplaytime,description,...,Book,Music,Environmental,Pike and Shot,Religious,Video Game Theme,Mature / Adult,Vietnam War,Fan Expansion,Korean War
0,1,Die Macher,1986.0,"[Economic, Negotiation, Political]",3.0,5.0,14.0,240.0,240.0,Die Macher is a game about seven sequential po...,...,0,0,0,0,0,0,0,0,0,0
1,2,Dragonmaster,1981.0,"[Card Game, Fantasy]",3.0,4.0,12.0,30.0,30.0,Dragonmaster is a trick-taking card game based...,...,0,0,0,0,0,0,0,0,0,0
2,3,Samouraï,1998.0,"[Abstract Strategy, Medieval]",2.0,4.0,10.0,30.0,60.0,Samurai is set in medieval Japan. Players comp...,...,0,0,0,0,0,0,0,0,0,0
3,4,Tal der Könige,1992.0,[Ancient],2.0,4.0,12.0,60.0,60.0,When you see the triangular box and the luxuri...,...,0,0,0,0,0,0,0,0,0,0
4,5,Acquire,1963.0,"[Economic, Territory Building]",2.0,6.0,12.0,90.0,90.0,"In Acquire, each player strategically invests ...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186108,205878,Multigenre horror RPG inSANe,NaN,[],NaN,NaN,NaN,NaN,NaN,From the back of the book:<br/><br/>The darkne...,...,0,0,0,0,0,0,0,0,0,0
186109,205879,Dakardsmyr: Im Nebel der Myrkansümpfe,NaN,[],NaN,NaN,NaN,NaN,NaN,<br/>\t\t\t\t<br/>\t\t\t\t\tPublisher blurbTie...,...,0,0,0,0,0,0,0,0,0,0
186110,205880,Dead Loop,NaN,[],NaN,NaN,NaN,NaN,NaN,From the back of the book:<br/><br/>The traged...,...,0,0,0,0,0,0,0,0,0,0
186111,205881,Villa Diodati,NaN,[],NaN,NaN,NaN,NaN,NaN,From the back of the book:<br/><br/>Adventure ...,...,0,0,0,0,0,0,0,0,0,0


In [16]:
for index, categories in enumerate(board_game_set['boardgamecategory']):
    for category in categories:
        board_game_set.loc[index, category] = 1

In [17]:
pd.set_option('display.max_columns', None)
board_game_set

,@objectid,name,yearpublished,boardgamecategory,minplayers,maxplayers,age,minplaytime,maxplaytime,description,image,Economic,Negotiation,Political,Card Game,Fantasy,Abstract Strategy,Medieval,Ancient,Territory Building,Civilization,Nautical,Childrens Game,City Building,Exploration,Travel,Farming,Mythology,Bluffing,Science Fiction,Collectible Components,Dice,Fighting,Print & Play,Maze,Miniatures,Racing,American West,Wargame,Adventure,Space Exploration,Renaissance,Modern Warfare,Humor,Electronic,Horror,Novel-based,Deduction,Word Game,Aviation / Flight,Movies / TV / Radio theme,Party Game,Memory,Math,Puzzle,Real-time,Trivia,Industry / Manufacturing,World War II,Expansion for Base-game,American Civil War,Age of Reason,American Indian Wars,World War I,Trains,Animals,Pirates,Murder/Mystery,Transportation,Prehistoric,Sports,Action / Dexterity,Game System,Number,Spies/Secret Agents,Medical,Mafia,Zombies,Comic Book / Strip,Arabian,Napoleonic,Civil War,Educational,Post-Napoleonic,American Revolutionary War,Book,Music,Environmental,Pike and Shot,Religious,Video Game Theme,Mature / Adult,Vietnam War,Fan Expansion,Korean War
0,1,Die Macher,1986.0,"[Economic, Negotiation, Political]",3.0,5.0,14.0,240.0,240.0,Die Macher is a game about seven sequential po...,https://cf.geekdo-images.com/rpwCZAjYLD940NWwP...,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,Dragonmaster,1981.0,"[Card Game, Fantasy]",3.0,4.0,12.0,30.0,30.0,Dragonmaster is a trick-taking card game based...,https://cf.geekdo-images.com/oQYhaJx5Lg3KcGis2...,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,Samouraï,1998.0,"[Abstract Strategy, Medieval]",2.0,4.0,10.0,30.0,60.0,Samurai is set in medieval Japan. Players comp...,https://cf.geekdo-images.com/o9-sNXmFS_TLAb7Zl...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,Tal der Könige,1992.0,[Ancient],2.0,4.0,12.0,60.0,60.0,When you see the triangular box and the luxuri...,https://cf.geekdo-images.com/nYiYhUlatT2DpyXaJ...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,Acquire,1963.0,"[Economic, Territory Building]",2.0,6.0,12.0,90.0,90.0,"In Acquire, each player strategically invests ...",https://cf.geekdo-images.com/FfguJeknahk88vKT7...,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186108,205878,Multigenre horror RPG inSANe,NaN,[],NaN,NaN,NaN,NaN,NaN,From the back of the book:<br/><br/>The darkne...,https://cf.geekdo-images.com/CQ_ikVzPHQJbWRnAE...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
186109,205879,Dakardsmyr: Im Nebel der Myrkansümpfe,NaN,[],NaN,NaN,NaN,NaN,NaN,<br/>\t\t\t\t<br/>\t\t\t\t\tPublisher blurbTie...,https://cf.geekdo-images.com/GLeL_Fune7e1p7dDU...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
186110,205880,Dead Loop,NaN,[],NaN,NaN,NaN,NaN,NaN,From the back of the book:<br/><br/>The trage

In [21]:
board_game_set.drop(['description', 'image', 'boardgamecategory'], axis=1)

,@objectid,name,yearpublished,minplayers,maxplayers,age,minplaytime,maxplaytime,Economic,Negotiation,Political,Card Game,Fantasy,Abstract Strategy,Medieval,Ancient,Territory Building,Civilization,Nautical,Childrens Game,City Building,Exploration,Travel,Farming,Mythology,Bluffing,Science Fiction,Collectible Components,Dice,Fighting,Print & Play,Maze,Miniatures,Racing,American West,Wargame,Adventure,Space Exploration,Renaissance,Modern Warfare,Humor,Electronic,Horror,Novel-based,Deduction,Word Game,Aviation / Flight,Movies / TV / Radio theme,Party Game,Memory,Math,Puzzle,Real-time,Trivia,Industry / Manufacturing,World War II,Expansion for Base-game,American Civil War,Age of Reason,American Indian Wars,World War I,Trains,Animals,Pirates,Murder/Mystery,Transportation,Prehistoric,Sports,Action / Dexterity,Game System,Number,Spies/Secret Agents,Medical,Mafia,Zombies,Comic Book / Strip,Arabian,Napoleonic,Civil War,Educational,Post-Napoleonic,American Revolutionary War,Book,Music,Environmental,Pike and Shot,Religious,Video Game Theme,Mature / Adult,Vietnam War,Fan Expansion,Korean War
0,1,Die Macher,1986.0,3.0,5.0,14.0,240.0,240.0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,Dragonmaster,1981.0,3.0,4.0,12.0,30.0,30.0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,Samouraï,1998.0,2.0,4.0,10.0,30.0,60.0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,Tal der Könige,1992.0,2.0,4.0,12.0,60.0,60.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,Acquire,1963.0,2.0,6.0,12.0,90.0,90.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186108,205878,Multigenre horror RPG inSANe,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
186109,205879,Dakardsmyr: Im Nebel der Myrkansümpfe,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
186110,205880,Dead Loop,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
186111,205881,Villa Diodati,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
def replace_nan_values(labels: list):
    for label in labels:
        for index, value in enumerate(board_game_set[label]):
            if np.isnan(value):
                board_game_set.loc[index] = 0